In [ ]:
from robot import robot
from measurer import map_measurer
from keypoints_map import keypoints_map

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
from IPython import display
import time
from matplotlib.lines import Line2D
    
%matplotlib inline

In [ ]:
class ParticleFilter(object):
    def __init__(self, measurer, num_particles=100):
        pos = np.random.normal(scale=10, size=(num_particles, 2))
        yaw = np.random.uniform(low=0, high=2 * np.pi, size=num_particles)
        vel = np.random.normal(scale=0.5, size=(num_particles, 2))
        
        # each particle is 2d position, 2d velocity and robot orientation (yaw)
        self.particles = np.hstack((pos, vel, yaw[:, np.newaxis]))
        self.weights = np.ones(num_particles)
        
        self.measurer_ = measurer
        # how large penalty for bad particles is
        self.ll_ratio_ = 0.1
    def move(self, dt):
        #TODO move particles according to dynamic model 
        # x' = x + dt * v
        # v' = v
        # yaw' = yaw
        # and apply random (normal) noise to each component
        # see e. g. Robot.move
        self.particles = None
        
        # TODO: what amplitude of noise should we use for position velocity and yaw?
        pos_noise = np.random.normal(scale=None, size=(num_particles, 2))
        yaw_noise = np.random.normal(scale=None, size=num_particles)
        vel_noise = np.random.normal(scale=None, size=(num_particles, 2))
        
        noise = np.hstack((pos_noise, vel_noise, yaw_noise[:, np.newaxis]))
        
        self.particles += noise
        
    def update(self, measure):
        for p_id, p in enumerate(self.particles):
            world_seen_from_particle = self.measurer_.measure(p[-1], p[:2])
            # TODO compose log-likelihood of observation given state
            # say, p(z | x) is normal
            # observation here is vector of coordinates of keypoints seen from robot
            
            ll = None
            ll = np.exp(-self.ll_ratio_ * ll)
            self.weights[p_id] = ll
    
    def measure_pos(self):
        # TODO How to measure expected position, given set of particles with a weight 
        return None
    
    def measure_yaw(self):
        # TODO how to measure particle orientation
        # hint it is not a good idea to average yaw directly (note 2pi modulus), better to get average of 
        # sin(yaw) and cos(yaw)
        return None
    
    def resample(self):
        # TODO Implement resampling
        # choose len(self.particles) particles to keep
        # with probabilities proportional to self.weights
        # hint use np.random.choise to get indecies of particles to keep
        choose = None
        self.particles = self.particles[choose]
        self.weights[:] = 1
        
    def draw_particles(self, ax):
        for p in self.particles:
            ax.arrow(p[0], p[1], np.sin(p[-1]), np.cos(p[-1]),
                     head_width=0.05, head_length=0.1, fc='k', ec='k', alpha=0.05)
            
            

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.subplot(111, aspect='equal')

keypoints = keypoints_map.KeypointsMap()
r = robot.Robot(
    pos=np.array((-7, 5)),
    vel=np.array((0.5, 0)),
    yaw=np.pi / 4,
    odometer_noise_x=0.1, odometer_noise_y=0.1)

m = map_measurer.MapMeasurer(keypoints)
pf = ParticleFilter(m, 300)

dt = 0.1

custom_lines = [Line2D([0], [0], color='red', lw=4),
                Line2D([0], [0], color='black', lw=4),
                Line2D([0], [0], color='green', lw=4)]

for i in range(200):
    ax.clear()
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    
    ax.legend(custom_lines, ['PF estimate', 'Particles', 'True robot position'])
    
    r.move(dt)
    # some funny rotation of our robot
    r.state_[-1] += 0.1
    measure = m.measure(r.state_[-1], r.state_[:2])
    
    #TODO: move particles
    #      update with measurements
    pos_hat = pf.measure_pos()
    yaw_hat = pf.measure_yaw()
    # TODO: resample
    
    pf.draw_particles(ax)
    ax.scatter(r.state_[0], r.state_[1], marker='x', color='green')
    ax.arrow(r.state_[0], r.state_[1], np.sin(r.state_[-1]), np.cos(r.state_[-1]),
                     head_width=0.05, head_length=0.1, fc='g', ec='g', alpha=1)
    
    ax.scatter(pos_hat[0], pos_hat[1], marker='o', color='red')
    ax.arrow(pos_hat[0], pos_hat[1], np.sin(yaw_hat), np.cos(yaw_hat),
                     head_width=0.05, head_length=0.1, fc='r', ec='r', alpha=1)
    
             
    display.clear_output(wait=True)
    display.display(plt.gcf())
    
    
display.clear_output(wait=True)
